<img src="images/MetroloJA_logo_white.png">

In [1]:
# Import
from metroloja_lib import coreg_analyze as ca
from metroloja_lib import psf_analyze as pa
import tkinter as tk
import os
import zipfile
from pathlib import Path

from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets
from IPython.display import display
from ipyfilechooser import FileChooser

In [2]:
starting_directory = '.'
chooser = FileChooser(starting_directory)
chooser.title = '<b>Select your input file</b>'
display(chooser)

FileChooser(path='/home/jovyan', filename='', title='<b>Select your input file</b>', show_hidden=False, select…

In [4]:
global folder_path
folder_path = os.path.splitext(chooser.selected)[0]

my_file = Path(chooser.selected_filename)
if my_file.is_file():
    with zipfile.ZipFile(chooser.selected_filename, 'r') as zip_ref:
        zip_ref.extractall('.')

    os.remove(chooser.selected_filename)

In [5]:
def start(b):
    a = ['Resolution','Coregistration','Laser Power','Field Homogeneity']
    N_buttons=4
    buttons_analyze_type=[widgets.Button(description = i) for i in a]
    outputs=widgets.HBox([items for items in buttons_analyze_type])
    display(outputs)
    
    
    button_measurement_choice = widgets.Button(description="OK!")
    output = widgets.Output()
    display(button_measurement_choice, output)
    
    button_measurement_choice.disabled = True
    
    
    
    def on_button_clicked1(x):
        global which_button_is_selected
        which_button_is_selected = x.description
        button_measurement_choice.disabled = False
        #print(which_button_is_selected)
    
    
    
    
    def on_button_clicked2(b):
        with output:
            print(f"Analyze type : {which_button_is_selected}")

            if which_button_is_selected  != None:
                if which_button_is_selected == 'Resolution': # Résolution part
                    print('OK PSF')

                    Input_folder = pa.select_folder(folder_path) # Select input folder
                    print('\n')
                    Processed_folder = pa.processed_path(Input_folder[0], Input_folder[1]) # Find MetroloJ_QC Processed folder
                    DF = pa.convert_to_df(Processed_folder, Input_folder[1], Input_folder[2], Input_folder[3], Input_folder[4], Input_folder[0]) # Creat dataframe containing all data
                    # DF[0] => dataframe for XYZ data
                    # DF[1] => dataframe for SBR data
                    DF_XYZ_stat = pa.XYZ_stats(DF[0]) # Stat for XYZ data
                    DF_SBR_stat = pa.SBR_stats(DF[1]) # Stat for SBR data

                    #########################################
                    # 
                    
                    
                    button_boxplot = widgets.Button(description="Show Boxplot!")
                    output2 = widgets.Output()
                    button_boxplot.layout.visibility = 'hidden'

                    param_selected = pa.select_param(button_boxplot)
                    
                    def on_button_clicked3(x):
                        pa.display_selected_plot(param_selected, folder_path, DF[0], DF[1], DF_XYZ_stat, DF_SBR_stat[0], DF_SBR_stat[1])

                    
                    button_boxplot.on_click(on_button_clicked3)
                    display(button_boxplot, output2)



                elif which_button_is_selected == 'Coregistration': # Coregistration part
                    print('OK Coreg')

                    Input_folder = ca.select_folder() # Select input folder
                    print('\n')
                    Processed_folder = ca.processed_path(Input_folder[0], Input_folder[1]) # Find MetroloJ_QC Processed folder
                    DF = ca.convert_to_df(Processed_folder, Input_folder[1], Input_folder[2], Input_folder[3], Input_folder[4], Input_folder[0]) # Creat dataframe containing all data
                    DF_stat = ca.coreg_stats(DF[0], DF[1]) # Dataframe for all stat
                    print('\n')
                    param_selected = ca.select_param() # Select analyze parameters to plot
                    print('\n')
                    date_list = DF[1]['Date'].tolist()
                    ca.display_selected_plot(param_selected[1], param_selected[0], DF[0], DF_stat[0], Input_folder[0], DF_stat[1], DF[2], date_list)

    

    for i in range(N_buttons):
        buttons_analyze_type[i].on_click(on_button_clicked1)


    button_measurement_choice.on_click(on_button_clicked2)


In [6]:
button = widgets.Button(description="Click to Start !", style=dict(font_weight='bold',
                                                                  button_color = 'lightgreen'), 
                        layout = widgets.Layout(width = '25%', height = '80px'))
output = widgets.Output()

box_layout = widgets.Layout(display='flex', flex_flow='column',
                            align_items='center')
box = widgets.HBox(children=[button],layout=box_layout)

display(box, output)

button.on_click(start)

Output()

Button(description='OK!', style=ButtonStyle())

Output()